# <center>Graph-Based Approach for Item Recommendations<center>

## Introduction:

The purpose of this approach is to convert the transactional data into a weighted graph, and mine it to find suitable substitutes for any given product. Based on our research, [GraphSWAG](https://arxiv.org/pdf/1911.10232.pdf) is a good algorithm for mining transactional graphs.

GraphSWAG is a Graph Convolutional Network (GCN) developed by researchers at Target that combines sales data, product descriptions and product images to generate node embeddings for large graphs. Since we did not have access to the data used by Target, we had to make modifications to the algorithm and try to implement it on our own.

<img src="notebook_image/shopping_graph.png" alt="shopping_graph" style="width: 325px;"/>

## 0. Importing Libraries:

In [19]:
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import collections
from datetime import datetime as dt
from datetime import date
from scipy.sparse import csr_matrix, load_npz, save_npz
from sklearn.neighbors import NearestNeighbors as NN
from numpy.random import choice
import gensim
from gensim.models import Word2Vec
from IPython.core.display import display, HTML

## 1. Importing Datasets:

In [20]:
# Transactions

ncr_transactions =  pd.read_csv('../raw_data/ncr/items_transactions.csv')
ncr_transactions.head()

,global_transaction_id,item_id,dept_num,qty_sold,item_price,qty_is_weight,ticket_num,date,time_scanned
0,0,4889,6,3,99,0,2527,2020-07-01,07:03:30
1,0,3125,6,460,429,1,2527,2020-07-01,07:03:34
2,1,5,20,4,100,0,2528,2020-07-01,07:04:22
3,1,2,20,6,100,0,2528,2020-07-01,07:04:25
4,2,7013201045,1,1,299,0,2529,2020-07-01,07:05:06


In [21]:
# Items

ncr_items =  pd.read_csv('../raw_data/ncr/items_descriptions.csv')
ncr_items.head()

,item_id,description,ecomm_description,category,item_type,upc
0,1,PAN DULCE SENCILLO,"Mexican Sweet Bread/Pan Dulce Mexicano, 1 Count",20101020,0,10
1,2,BOLILLO FRENCH ROLLS,"Bolillo, French Rolls, 1 Count",20101210,0,20
2,3,BOLILLO QUESO/CHILE JALAP,"Jalapeño and Cheese Bolillo, 1 Count",20101210,0,30
3,4,EMPANADA,NaN,20101020,0,40
4,5,MIni Bolillo,"BOLILLO SMALL, 2 OZ",20101210,0,50


## 2. Data Pre-processing

### 2.1 Removing nonrelevant products

In [22]:
# Removing product id's from nonrelevant products, such as paper bags, plastic bag tax refunds, etc...

idsToRemove = [9492206955, 5555, 75,7501,7502,7503,7504,7505,7506,7507,7508,7509,7510,7511,7512,7513,7514,7515,7516,7517,7518,
7519,7520,7521,7522,7523,7524,7525,7526,7527,7528,7529,7530,7531,7532,7533,7535,7536,7537,7538,7539,7540,
7541,7542,7543,7544,7545,7546,7547,7548,7549,7550,8771,8772,9999910012]

ncr_items = ncr_items[~ncr_items.item_id.isin(idsToRemove)]
ncr_items = ncr_items.replace(np.nan, '', regex=True)

ncr_items["description"] = ncr_items["description"].map(str) + ' ' + ncr_items["ecomm_description"].map(str)
ncr_items["description"] = ncr_items["description"].str.rstrip(' ')
ncr_items["description"] = ncr_items["description"].str.lower().str.replace('/',' ').str.replace('-',' ').replace(r'[^A-Za-z0-9. ]+', '', regex=True)

ncr_transactions = ncr_transactions[~ncr_transactions.item_id.isin(idsToRemove)]
display(ncr_items.head())
display(ncr_transactions.head())

,item_id,description,ecomm_description,category,item_type,upc
0,1,pan dulce sencillo mexican sweet bread pan dul...,"Mexican Sweet Bread/Pan Dulce Mexicano, 1 Count",20101020,0,10
1,2,bolillo french rolls bolillo french rolls 1 count,"Bolillo, French Rolls, 1 Count",20101210,0,20
2,3,bolillo queso chile jalap jalapeo and cheese b...,"Jalapeño and Cheese Bolillo, 1 Count",20101210,0,30
3,4,empanada,,20101020,0,40
4,5,mini bolillo bolillo small 2 oz,"BOLILLO SMALL, 2 OZ",20101210,0,50


,global_transaction_id,item_id,dept_num,qty_sold,item_price,qty_is_weight,ticket_num,date,time_scanned
0,0,4889,6,3,99,0,2527,2020-07-01,07:03:30
1,0,3125,6,460,429,1,2527,2020-07-01,07:03:34
2,1,5,20,4,100,0,2528,2020-07-01,07:04:22
3,1,2,20,6,100,0,2528,2020-07-01,07:04:25
4,2,7013201045,1,1,299,0,2529,2020-07-01,07:05:06


### 2.2 Saving the item ids and item names

In [11]:
# Separating the item ids and item names as single arrays

item_ids = np.asarray(ncr_items["item_id"])
item_names = np.asarray(ncr_items["description"].str.split())

# Saving the arrays to be quickly loaded if needed

np.save('../raw_data/model_outputs/item_ids.npy', item_ids)
np.save('../raw_data/model_outputs/item_names.npy', item_names)

In [23]:
# Loading pre-saved item ids and item names arrays

item_ids = np.load('../raw_data/model_outputs/item_ids.npy', allow_pickle=True)
item_names = np.load('../raw_data/model_outputs/item_names.npy', allow_pickle=True)

# 3. Generating the Weighted Graph

**Graph Structure:**
- **Nodes:** Individual Products
    - Non-food items such as plastic bags were removed
    - Items= ids without names will be removed
- **Edges:** Weighted Pairwise Co-Occurrence
    - Weights are calculated by adding co-occurrent transactions adjusted by **time-decay** and transforming them to a [0,1] range using the **arctan** function
 
<br><br>  
**Edge Weighting Function:**
<img src="notebook_image/weighting_function.png" alt="weighting_function" style="width: 350px;"/>
<br><br>  

### 3.1 Converting the transactional dataset to a list of weighted edges

In [49]:
graph_dict = defaultdict(int)
transaction_ids = ncr_transactions.global_transaction_id.unique()
today = date.today()

# Looping through unique transaction ids
for id in transaction_ids:
    if id % 10000 == 0 or id == 0 or id == len(transaction_ids):
        print('Transaction ' + str(id + 1) + ' of ' + str(len(transaction_ids)))
    
    # Looping through each unique transaction
    transaction = ncr_transactions[ncr_transactions.global_transaction_id == id]
    for i in range(len(transaction) - 1):
        for j in range(i,len(transaction)):
            pair = (transaction.iloc[i].item_id,transaction.iloc[j].item_id)
            if not isinstance(pair,int):
                
                # Checking if both product ids are different
                if len(set(pair)) == len(pair):
                    
                    # Adding co-occurance weight, applying time decay 
                    transaction_date = dt.strptime(transaction.iloc[i].date, '%Y-%m-%d').date()
                    delta = today - transaction_date
                    graph_dict[pair] += 1/delta.days

Transaction 1 of 123078
Transaction 10001 of 123078
Transaction 20001 of 123078
Transaction 30001 of 123078
Transaction 40001 of 123078
Transaction 50001 of 123078
Transaction 60001 of 123078
Transaction 70001 of 123078
Transaction 80001 of 123078
Transaction 90001 of 123078
Transaction 100001 of 123078
Transaction 110001 of 123078
Transaction 120001 of 123078
Transaction 123079 of 123078


In [16]:
# Converting the graph dictionary as a list of lists
graph_list = []

for pair in graph_dict:
    if not isinstance(pair,int):
        row = list(pair)
        row.append(graph_dict[pair])
        graph_list.append(row)

In [18]:
# Converting the graph list into a Pandas DataFrame
graph_df = pd.DataFrame(data = graph_list, columns=['product_a', 'product_b', 'weight'])

# Grouping the DataFrame by product pairs
graph_df = graph_df.groupby(['product_a', 'product_b'])[['weight']].sum()

# Normalizing the graph weights
weight_sums = graph_df.groupby('product_a')['weight'].sum()
graph_df.weight = graph_df.weight / weight_sums
graph_df = graph_df.reset_index()


# Applying the Arctan function to the weight values
graph_df['weight'] = np.arctan(graph_df['weight'])

graph_df.head()

,product_a,product_b,weight
0,1,2,0.026733
1,1,3,0.001332
2,1,5,0.005448
3,1,6,0.000069
4,1,9,0.000080


In [19]:
# Saving the edge list DataFrame as a CSV file
graph_df.to_csv('../raw_data/model_outputs/graph_edges.csv', index=False)

In [9]:
# Loading the edge list graph as a Pandas DataFrame
graph_df = pd.read_csv('../raw_data/model_outputs/graph_edges.csv')

### 3.2 Converting the edge list into an adjacency matrix

In [11]:
# Creating a NetworkX object from the edge list DataFrame
graph = nx.from_pandas_edgelist(graph_df, 'product_a', 'product_b', edge_attr='weight')
graph.name = 'Product Transactions Graph'

In [12]:
# Printing information about the graph
graph_info = nx.info(graph)
graph_info = graph_info.replace('\n','<br>')
display(HTML(graph_info))

In [41]:
# Exporting the graph as a numpy adjacency matrix
adj_matrix = nx.to_numpy_matrix(graph)
adj_matrix.shape

(10047, 10047)

In [42]:
# Saving the adjacency matrix as a npy file
np.save('../raw_data/model_outputs/adj_matrix.npy', adj_matrix)

In [4]:
adj_matrix = np.load('../raw_data/model_outputs/adj_matrix.npy')

In [5]:
# Converting the adjacency matrix to a CSR sparse matrix to save memory
sparse_adj_matrix = csr_matrix(adj_matrix)
sparse_adj_matrix.shape

(10047, 10047)

In [6]:
# Saving the sparse adjacency matrix as an npz file
save_npz('../raw_data/model_outputs/sparse_adj_matrix.npz', sparse_adj_matrix)

In [7]:
sparse_adj_matrix = load_npz("../raw_data/model_outputs/sparse_adj_matrix.npz")
sparse_adj_matrix.shape

(10047, 10047)

### 3.3 Loading the Word Embedding Vectors generated using Word2Vec

In [15]:
item_vectors = np.load("../raw_data/model_outputs/item_vectors.npy")
item_vectors.shape

(48496, 200)

### 3.4 Matching the Product IDs in the Edge Graph and Word Vectors

In [24]:
# Dropping word embedding vectors from item ids that were not present in the transactional dataset

unique_ids = sorted(graph_df['product_a'].append(graph_df['product_b']).unique())

vector_df = pd.DataFrame(data = item_ids, columns = ['item_id'])
vector_df['vector'] = [item_vectors[i] for i in range(len(item_vectors))]
vector_df = vector_df[vector_df.item_id.isin(unique_ids)].reset_index(drop='True')

In [27]:
# Dropping graph edges with product ids that were not present in the item names dataset

vector_unique_ids = vector_df.item_id.unique()

graph_df = graph_df[graph_df.product_a.isin(vector_unique_ids)]
graph_df = graph_df[graph_df.product_b.isin(vector_unique_ids)]

In [28]:
# Saving the cleaned edge list DataFrame as a CSV file
graph_df.to_csv('../raw_data/model_outputs/graph_edges_clean.csv', index=False)

In [29]:
# Re-creating the NX graph object
graph = nx.from_pandas_edgelist(graph_df, 'product_a', 'product_b', edge_attr='weight')

# Re-creating the adjacency matrix
adj_matrix = nx.to_numpy_matrix(graph)

# Converting it to a CSR sparse matrix
sparse_adj_matrix = csr_matrix(adj_matrix)
sparse_adj_matrix.shape

(9722, 9722)

In [30]:
# Saving the cleaned sparse adjacency matrix as an npz file
save_npz('../raw_data/model_outputs/sparse_adj_matrix_clean.npz', sparse_adj_matrix)

## 4. Building the Graph Convolutional Network (GCN)

**Graph Convolutional Network Steps:**
- Sampling:
    - K neighbors of each node are sampled before each layer.
    - A beta hyperparameter assign a higher or lower importance to the edge weights when generating sampling probabilities.
- Aggregation:
    - The feature vectors of all sampled neighbors of a node a combined with some aggregation function.
    - We used the Mean aggregator, similarly to the Target paper.
- Concatenation:
    - Each aggregated feature vector is concatenated to the output of the previous layer before being input to the next layer.
- Non-Linear Activation Function:
    - All hidden layers used the ReLu activation funciton
    - The final layer used the Softmax activation funciton
- Final Output:
    - 200-long feature vectors embedding each node of the graph.

**Graph Convolutional Network (GCN) Architecture**:
<img src="notebook_image/gcn.png" alt="gcn" style="width: 800px;"/>

### 4.1 Sampling & Aggregation

**Sampling and Aggregation Algorithms**:
<br>
<img src="notebook_image/s_a.png" alt="s_a" style="width: 750px;"/>
<br>

In [45]:
def sampling_neighbors(a_series, beta, k):
    all_weights = a_series.to_numpy()
    nonzero_indices = np.nonzero(all_weights)[0]
    nonzero_weights = np.power(all_weights[np.nonzero(all_weights)],beta) * k
    nonzero_weights /= sum(nonzero_weights)

    neighbors = np.random.choice(nonzero_indices, 
                                 min(k,len(np.nonzero(nonzero_weights)[0])), 
                                 p=nonzero_weights, replace=False)
    weights = all_weights[neighbors]
    return [(neighbors[i], weights[i]) for i in range(0, len(neighbors))] 

def expand_graph(graph_df):
    graph_df["u_adj_idx"] = np.arange(sparse_adj_matrix.shape[0])
    graph_df = graph_df.explode("neighbors and weights")
    graph_df["v_adj_idx"] = graph_df["neighbors and weights"].apply(lambda x: x[0])
    graph_df["weight"] = graph_df["neighbors and weights"].apply(lambda x: x[1])
    return graph_df[["u_adj_idx","v_adj_idx","weight"]]

def add_vectors_column(graph_df, vectors_df):
    graph_df = graph_df.merge(vectors_df.item_id, how = 'inner', left_on = 'u_adj_idx', right_index = True)
    graph_df = graph_df.merge(vectors_df, how = 'inner', left_on = 'v_adj_idx', right_index = True)
    graph_df.columns = ['u_adj_idx', 'v_adj_idx', 'weight', 'u_id', 'v_id', 'vector']
    graph_df = graph_df.sort_values(by=['u_adj_idx', 'v_adj_idx'])
    return graph_df


def sampling(sparse_adj_matrix, vectors_df, beta = 10, k = 10):
    pd.options.mode.chained_assignment = None
    w = pd.DataFrame(sparse_adj_matrix.toarray())
    w["neighbors and weights"] = w.apply(lambda a_series: sampling_neighbors(a_series, beta, k), axis=1)
    return expand_graph(w[["neighbors and weights"]])

def sampling_and_aggregating(sparse_adj_matrix, prev_vectors, beta = 10, k = 10, gamma = 10):
    
    this_graph = sampling(sparse_adj_matrix, vector_df)
    this_graph = add_vectors_column(this_graph, prev_vectors)

    aggregated = this_graph.groupby('u_adj_idx').vector.apply(lambda g: np.mean(g.values.tolist(), axis=0))
    aggregated = aggregated.apply(lambda g: g/np.linalg.norm(g))

    aggregated = [prev_vectors.vector.iloc[i].tolist() + aggregated.iloc[i].tolist() for i in range(len(prev_vectors))]
    new_vectors = prev_vectors.copy()
    new_vectors['vector'] = aggregated
    
    return np.matrix(aggregated, dtype=float)

### 4.2 Building GCN Layers and Activation Functions

In [46]:
from scipy.special import softmax

def prepare_h0(vector_df):
    h0 = vector_df.copy()
    h0['vector'] = h0['vector'].apply(lambda g: g/np.linalg.norm(g))
    return h0

def relu(h):
    return np.maximum(h, 0)

def gcn_layer(layer_input, W, vector_df, activation='relu'):
    h = vector_df.copy()
    if activation == 'relu':
        act_matrix = np.asarray(relu(layer_input * W))       
    elif activation == 'softmax':
        act_matrix  = np.asarray(softmax(layer_input * W))
    
    h['vector'] = [np.array(row) for row in act_matrix]
    h['vector'] = h['vector'].apply(lambda g: g/np.linalg.norm(g))
    return h

### 4.3 Get Product Recommendations using the Nearest Neighbors Aglorithm

In [47]:
def get_nearest_neighbors(z, n_neighbors=5):
    
    knn_model = NN(n_neighbors=n_neighbors)
    knn_model.fit(z.vector.tolist())
    
    return knn_model.kneighbors_graph(z.vector.tolist()).toarray()

In [48]:
def print_recommendations_by_product(nn_matrix, vector_df, items_df, item_idx):
    item = items_df[items_df['item_id'] == vector_df.iloc[item_idx].item_id].description.values[0]
    print("Substitutes to \"" + item + "\":")
    print(items_df[items_df['item_id'].isin(vector_df.iloc[np.where(nn_matrix[item_idx])[0].tolist()].item_id.values)].description)

### 4.4 Initial trail run of the GCN

#### 4.4.1 Randomly generating initial GCN weight parameters

In [49]:
W_1 = np.random.normal(
    loc=0, scale=1, size=(400, 200))
W_2 = np.random.normal(
    loc=0, size=(400, 200))
W_3 = np.random.normal(
    loc=0, size=(400, 20))

#### 4.4.2 Inputting data through the GCN

In [50]:
h0 = prepare_h0(vector_df)

layer_input = sampling_and_aggregating(sparse_adj_matrix, h0)
h1 = gcn_layer(layer_input, W_1, vector_df)

layer_input = sampling_and_aggregating(sparse_adj_matrix, h1)
h2 = gcn_layer(layer_input, W_2, vector_df)

layer_input = sampling_and_aggregating(sparse_adj_matrix, h2)
z = gcn_layer(layer_input, W_3, vector_df, 'softmax')

nn_matrix = get_nearest_neighbors(z,6)

#### 4.4.2 Printing some recommendation examples

In [51]:
for _ in range(5):
    rand_idx = np.random.randint(len(vector_df))
    print_recommendations_by_product(nn_matrix, vector_df, ncr_items, rand_idx)
    print()

Substitutes to "coors light coors light 30pk 12 ounce":
3564                            lets celebrate arrangement
8273     ah liquid 2 x conc clean burst arm  hammer liq...
8571     reeses peanut butter cups reeses peanut butter...
9964                  tide simply oxi refresh breeze liq d
15535    anthy small shells anthonys small shells 16 ounce
18626                coors light coors light 30pk 12 ounce
Name: description, dtype: object

Substitutes to "maiz cancha amarillo maiz cancha amarillo 15 oz":
4489     floridas nat oj w pulp floridas nat oj w pulp ...
9737     always quiltwd thin long always pads ultra thi...
11498    martinelli sparkling apl grape martinelli spar...
12996    yakisoba teriyaki maruchan teriyaki beef flavo...
28462    bistec de res para la parrilla beef steak gril...
38442      maiz cancha amarillo maiz cancha amarillo 15 oz
Name: description, dtype: object

Substitutes to "knorr chicken rice side knorr rice sides rice sides dish chicken 5.6 oz":
4862        

## 5.Training the Graph Convolutional Network (GCN)

### 5.1 The Loss Function

<img src="notebook_image/loss.png" alt="loss" style="width: 800px;"/>

In [ ]:
# inputs: 
# 1. item_id corresponding to node u (scalar) 
# 2. vector representations of all nodes Z (matrix formatted as a 2D numpy array)
# 3. k-means output (df4 above) (pandas dataframe)
def loss_function(item_id, Z, df4):
    
    # step 1: match item_id to row index (u)
    u = # not sure how to do this
    
    # step 2: set alpha 
    alpha = 0.5

    # step 3: go on random walk
    # u is the starting node
    p = np.zeros(T.shape[0])
    p[u] = 1
    p = p.reshape(-1,1)
    # define the random walk length, say 10
    walkLength = 10
    weights = []
    visited = []
    starting_node = u
    for k in range(walkLength):
        # evaluate the next state vector
        p = np.dot(T,p)
        # choose the node with higher probability as the visited node
        visited.append(np.argmax(p))
        weights.append(T[starting_node, np.argmax(p)])
        starting_node = np.argmax(p)
    # v is the node where you end up at the end of the random walk
    v=visited[-1]
    # calculate geometric mean of weights along random walk
    geometric_mean = stats.gmean(weights)
    
    # step 4: define sigmoid function
    def sigmoid(x):   
        z = 1/(1 + np.exp(-x)) 
    return z

    # step 5: calculate first term
    # assuming that row u of matrix Z is the vector representation for node u
    first_term = -(geometric_mean**alpha)*np.log(sigmoid(np.dot(Z[u,:],Z[v,:])))
    
    # step 6: calculate second term
    # recommended to use Q = 2, 3, 4, or 5 since we have a large dataset
    Q = 3 
    # get one positive sample and Q negative samples
    # positive sample is from u_cluster
    # negative samples are not from u_cluster
    
    # which cluster corresponding to node u?
    u_cluster = df4[df4["item_id"] == item_id]["cluster"].tolist()[0]
    
    positive_sampling_df = df4[df4["cluster"] == u_cluster]
    row = positive_sampling_df.sample()
    positive_item_id = row["item_id"].tolist()[0]
    
    negative_sampling_df = df4[df4["cluster"] != u_cluster]
    rows = negative_sampling_df.sample(n=3, replace=False)
    negative_item_ids = rows["item_id"].tolist()
    
    # need to get corresponding rows of Z to fill in below by matching item_ids to row indices
    positive_sample = np.log(sigmoid(np.dot(-Z[u,:],Z[,:])))
    negative_sample_1 = np.log(sigmoid(np.dot(-Z[u,:],Z[,:])))
    negative_sample_2 = np.log(sigmoid(np.dot(-Z[u,:],Z[,:])))
    negative_sample_3 = np.log(sigmoid(np.dot(-Z[u,:],Z[,:])))
    
    # then calculate expectation (this is just arithmetic mean in this case?)
    expectation = (positive_sample+negative_sample_1+negative_sample_2+negative_sample_3)/4
    second_term = Q*expectation
    
    # step 7: subtract second term from first term to get loss
    loss = first_term - second_term
    
    return loss

### 5.2 Implementing training via Backpropagation (Future Work)

The next step to finalize the GCN would be to actually implement a training function using the Loss Function shown above. The training procedure would utilize the ***Backpropagation Algorithm***, which could be implemented from "scratch" using only NumPy/SciPy functions, or with the help of Deep Learning libraries like **Apache MXNet**, **TensorFlow** or **Caffe**.

## 6. Potential Future Improvements

Some avenues for future work on our implementation are training Word2Vec, using more data, and tuning hyperparameters. We used Gensim's implementation of Word2Vec, which is pre-trained on the Google News dataset. As far as using more data, item descriptions and images could be used. The researchers at Target used these features in their implementation. We only used item names because we did not have item descriptions for most items, and we did not have any item images. If this data becomes available in the future, it can be incorporated into the model easily. Lastly, the following hyperparameters can be tuned: length of item vectors in Word2Vec, length of random walks, number of random walks, number of neighbors in the sampling algorithm, beta in the sampling algorithm, gamma in the aggregation algorithm, alpha in the loss function, number of layers in the network. Perhaps even better substitutes can be obtained by working in some or all of these areas.